In [ ]:
# code point access
ord("h")

In [36]:
import regex as re

In [ ]:
ord("😀")

In [ ]:
ord("হ")

In [ ]:
ord("hello")

In [ ]:
s = "নমস্কার (Hello in Bengali)"
c = [ord(x) for x in s]
c

In [ ]:
s.encode("utf-8")

In [ ]:
list(s.encode("utf-8"))

In [ ]:
# test is from https://www.reedbeta.com/blog/programmers-intro-to-unicode/
text = "Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception."
tokens = text.encode("utf-8")
tokens = list(map(int, tokens))
print(text)
print(len(text))
print(tokens)
print(len(tokens))


In [ ]:
chr(101), chr(32)

In [ ]:
def get_stat(ids):
    count = {}
    for pair in zip(ids, ids[1:]):
        count[pair] = count.get(pair, 0) + 1
    return count
stat = get_stat(tokens)
# print(stat)
# print(sorted(((v, k) for k, v in stat.items()), reverse=True))

In [ ]:
top_pair = max(stat, key=stat.get)
top_pair

In [ ]:
def merge(ids, pair, idx):
    newids = []
    i = 0
    while i < len(ids):
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i + 1] == pair[1]:
            newids.append(idx)
            i += 2
        else:
            newids.append(ids[i])
            i += 1
    return newids
tokens2 = merge(tokens, top_pair, 256)
print(tokens2)
print("Length: ", len(tokens2))

In [24]:
text = open('raw_text.txt', 'r').read()
tokens = text.encode("utf-8")
tokens = list(map(int, tokens))

In [25]:
def get_stat(ids):
    count = {}
    for pair in zip(ids, ids[1:]):
        count[pair] = count.get(pair, 0) + 1
    return count
stat = get_stat(tokens)

def merge(ids, pair, idx):
    newids = []
    i = 0
    while i < len(ids):
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i + 1] == pair[1]:
            newids.append(idx)
            i += 2
        else:
            newids.append(ids[i])
            i += 1
    return newids

##--
vocab_size = 276
num_merges = vocab_size - 256
ids = list(tokens)

merges = {}
for i in range(num_merges):
    count = get_stat(ids)
    top_pair = max(count, key=count.get)
    idx = 256 + i
    print(f"Merging {top_pair} into {idx}")
    ids = merge(ids, top_pair, idx)
    merges[top_pair] = idx



Merging (101, 32) into 256
Merging (105, 110) into 257
Merging (115, 32) into 258
Merging (116, 104) into 259
Merging (101, 114) into 260
Merging (99, 111) into 261
Merging (116, 32) into 262
Merging (226, 128) into 263
Merging (44, 32) into 264
Merging (97, 110) into 265
Merging (111, 114) into 266
Merging (100, 32) into 267
Merging (97, 114) into 268
Merging (101, 110) into 269
Merging (257, 103) into 270
Merging (261, 100) into 271
Merging (121, 32) into 272
Merging (46, 32) into 273
Merging (97, 108) into 274
Merging (259, 256) into 275


In [27]:
print(f"tokens length: {len(tokens)}")
print(f"ids length: {len(ids)}")
print(f"compression ratio: {len(tokens) / len(ids): .2f}")

tokens length: 24547
ids length: 19384
compression ratio:  1.27


In [32]:
vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]

def decode(ids):
    # given sequence tokens -> python str
    tokens = b"".join(vocab[id] for id in ids)
    text = tokens.decode("utf-8", errors="replace")
    return text

decode([128])

'�'

In [33]:
def encode(text):
    # given str -> seq of integers
    tokens = text.encode("utf-8")
    while len(tokens) >= 2:
        stats = get_stat(tokens)
        pair = min(stats, key=lambda p: merges.get(p, float("inf")))
        if pair not in merges:
            break
        idx = merges[pair]
        tokens = merge(tokens, pair, idx)
    return tokens

encode("Hello world")

[72, 101, 108, 108, 111, 32, 119, 266, 108, 100]

In [34]:
decode(encode("Hello world"))

'Hello world'

In [40]:
gpt2pat = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""")
print(re.findall(gpt2pat, "Hello world      you    "))

['Hello', ' world', '     ', ' you', '    ']


In [39]:
import tiktoken

gpt2_enc = tiktoken.get_encoding('gpt2')
print(gpt2_enc.encode("     Hello world"))

gpt4_enc = tiktoken.get_encoding('cl100k_base')
gpt4_enc.encode("     Hello world")

[220, 220, 220, 220, 18435, 995]


[257, 22691, 1917]

In [41]:
!wget https://openaipublic.blob.core.windows.net/gpt-2/models/1558M/vocab.bpe
!wget https://openaipublic.blob.core.windows.net/gpt-2/models/1558M/encoder.json

--2025-05-17 23:46:02--  https://openaipublic.blob.core.windows.net/gpt-2/models/1558M/vocab.bpe
Resolving openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)... 57.150.97.129
Connecting to openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)|57.150.97.129|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 456318 (446K) [application/octet-stream]
Saving to: ‘vocab.bpe’

vocab.bpe           100%[===================>] 445.62K   211KB/s    in 2.1s    

2025-05-17 23:46:06 (211 KB/s) - ‘vocab.bpe’ saved [456318/456318]

--2025-05-17 23:46:06--  https://openaipublic.blob.core.windows.net/gpt-2/models/1558M/encoder.json
Resolving openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)... 57.150.97.129
Connecting to openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)|57.150.97.129|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1042301 (1018K) [application/json]
Saving t

In [42]:
import os, json

with open('encoder.json', 'r') as f:
    encoder = json.load(f) # <--- ~equivalent to our "vocab"

with open('vocab.bpe', 'r', encoding="utf-8") as f:
    bpe_data = f.read()
bpe_merges = [tuple(merge_str.split()) for merge_str in bpe_data.split('\n')[1:-1]]
# ^---- ~equivalent to our "merges"

In [43]:
len(encoder)

50257

In [44]:
encoder['<|endoftext|>']

50256

KeyError: 'hellooo'